# Resume classifier

In [1]:
import cv2
import numpy as np
import imutils
import pytesseract
import pdf_to_image
import os
import time
import shutil

In [2]:
def isloate_text(filename):
    os.mkdir('pages')
    path = 'data/' + filename
    if filename[-3:] == 'pdf' or filename[-3:] == 'PDF':
        pdf_to_image.pdfToImages(path)
    else:
        img = cv2.imread(path)
        cv2.imwrite('pages/' + filename, img)
    i = 1
    for page in os.listdir('pages/'):
        path = 'pages/' + page
        rgb = cv2.imread(path)
        rgb = imutils.resize(rgb, width = int(rgb.shape[1]/4))
        small = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)

        _, bw = cv2.threshold(small, 0.0, 255.0, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 1))
        connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)

        contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        with open('headers.txt') as f:
            lines = f.read().upper().splitlines()

        header_cords = []

        for idx in range(len(contours)):
            x, y, w, h = cv2.boundingRect(contours[idx])
            cv2.rectangle(rgb, (x, y, x+w, y+h), (0, 0, 255), 2)
            try:
                cropped = rgb[y-5:y+h+5, x-5:x+w+5]
                text = pytesseract.image_to_string(cropped)
            except:
                continue
            text = text.lower()
            print(text)
            if text in lines or text[:-1] in lines:
                if text[len(text) - 1] != '.':
                    header_cords.append([x, y, x+w, y+h])
        cv2.imwrite('result/' + filename[:-4] + '-' + str(i) + '.jpg', rgb)
        i = i + 1
    shutil.rmtree('./pages/')
    return header_cords

In [3]:
total = 0
count = 0
for each in os.listdir('data/'):
    start = time.time()
    print('Working on: ' + each)
    isloate_text(each)
    total_time = time.time() - start
    print('Time taken: ' + str(total_time) + ' seconds')
    total = total + total_time
    count = count + 1
print('Total time taken: ' + str(total) + ' seconds')
print('Average time taken: ' + str(total/count) + ' seconds')

Working on: Cyber-Security-Resume-Example.jpg
Time taken: 14.455924034118652 seconds
Working on: Data-Analyst-Resume-Example-Template.jpg
Time taken: 2.392705202102661 seconds
Working on: data-scientist-resume-example-template.jpg
Time taken: 1.3082523345947266 seconds
Working on: Entry-Level-Software-Engineer-Resume-Sample-1.jpg
Time taken: 0.7675607204437256 seconds
Working on: Front-End-Developer-Resume-Example.jpg
Time taken: 5.034142255783081 seconds
Working on: Full-Stack-Developer-Resume-Example.jpg
Time taken: 1.2552680969238281 seconds
Working on: Help-Desk-Resume-Sample-Template.jpg
Time taken: 0.9814395904541016 seconds
Working on: Information-Technology-Resume-Example-Template.jpg
Time taken: 1.0663871765136719 seconds
Working on: IT-Project-Manager-Resume-Template.jpg
Time taken: 1.6520745754241943 seconds
Working on: Java-Developer-Resume-Sample-Template.jpg
Time taken: 1.4221851825714111 seconds
Working on: Net-Developer-Resume-Sample.jpg
Time taken: 1.0583956241607666 s

# Resume Parser  with GUI

In [4]:
import os
import flask
import pickle
import sys, fitz
from flask import Flask, request, url_for, render_template, send_from_directory
from itertools import *
import time
import random

import spacy
import sqlite3

In [5]:
UPLOAD_FOLDER = os.path.dirname(os.path.abspath("__file__")) + '/uploads/'

In [6]:
model_path = './model/model_2/'
model = spacy.load(model_path)

app = Flask(__name__, static_url_path="/static")
DIR_PATH = os.path.dirname(os.path.abspath("__file__"))
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [7]:
@app.route('/uploads/<filename>')
def uploaded_file(filename):
   return send_from_directory(app.config['DOWNLOAD_FOLDER'], filename, as_attachment=True)

@app.route('/', methods=['GET', 'POST'])
def main():
    if request.method == 'POST':
        f = request.files.getlist('files')
        result = []

        for file in f:
            if file:
                filename = file.filename
                file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))

                upload_dir = os.path.join(app.config['UPLOAD_FOLDER'], filename)
                
                doc = fitz.open(upload_dir)
                text = ""
                for page in doc:
                    text = text + str(page.getText())
                
                tx = "".join(text.split('\n'))
                doc = model(tx)
                # entities = {key: list(g) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)}
                entities = {key: list(set(map(lambda x: str(x), g))) for key, g in groupby(sorted(doc.ents, key=lambda x: x.label_), lambda x: x.label_)}

                result.append(entities)

        return render_template('result.html', result=result)

    return render_template('index.html')

In [ ]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/May/2021 11:09:34] "GET / HTTP/1.1" 200 -
mupdf: cannot recognize version marker
mupdf: invalid key in dict
mupdf: no objects found
[2021-05-13 11:10:01,193] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\hp\anaconda3\lib\site-packages\flask\